In [ ]:
%load_ext autoreload

In [2]:
import torch
import torchvision.datasets
import torch.optim as optim
import torch.functional
from pathlib import Path
i

In [26]:
%autoreload
from resnet import ResNet

[autoreload of resnet failed: Traceback (most recent call last):
  File "/software/ml/JupyterHub/conda-env-20191018-0822/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/software/ml/JupyterHub/conda-env-20191018-0822/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/software/ml/JupyterHub/conda-env-20191018-0822/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/software/ml/JupyterHub/conda-env-20191018-0822/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "/software/ml/JupyterHub/conda-env-20191018-0822/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    frame = next(frame_nfo.frame for frame_nfo in inspect.stack()
StopIteration
]


In [27]:
mnist = torchvision.datasets.MNIST(Path('mnist'), download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

In [44]:
train_loader = torch.utils.data.DataLoader(mnist, batch_size=1000, shuffle=True, num_workers=4, pin_memory=True)

In [45]:
model = ResNet(n_layers=4, n_classes=len(mnist.classes), n_input_channels=1, input_size=(mnist.train_data.shape[1], mnist.train_data.shape[2]))
model.cuda()

ResNet(
  (first_layer): Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (layers): ModuleList(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (activate): ReLU()
  (output_layer): Sequential(
    (0): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (1): Flatten()
    (2): Linear(in_features=12544, out_features=10, bias=True)
  )
)

In [46]:
optimizer = optim.SGD(model.parameters(), lr=3e-3,
                      momentum=0.9)

In [49]:
n_epochs = 20
criterion = torch.nn.CrossEntropyLoss()

In [50]:
for epoch_idx in range(0, n_epochs):
    
    epoch_losses = []    
    for batch_idx, (input, target) in enumerate(train_loader):
        optimizer.zero_grad()
        
        prediction = model(input.cuda())
        loss = criterion(prediction, target.cuda())
        epoch_losses.append(loss.item())
        
        loss.backward()
        optimizer.step()
        
    print("epoch", epoch_idx, "loss", torch.tensor(epoch_losses).mean().item())
        
        

epoch 0 loss 0.19441328942775726
epoch 1 loss 0.15757425129413605
epoch 2 loss 0.1265808641910553
epoch 3 loss 0.10514818131923676
epoch 4 loss 0.09189938008785248
epoch 5 loss 0.08091563731431961
epoch 6 loss 0.0744573250412941
epoch 7 loss 0.06809590011835098
epoch 8 loss 0.06382470577955246
epoch 9 loss 0.060749925673007965
epoch 10 loss 0.05767814815044403
epoch 11 loss 0.055968914180994034
epoch 12 loss 0.05382270738482475
epoch 13 loss 0.050836529582738876
epoch 14 loss 0.04933060333132744
epoch 15 loss 0.04756957292556763
epoch 16 loss 0.04640999063849449
epoch 17 loss 0.044726334512233734
epoch 18 loss 0.043298665434122086
epoch 19 loss 0.04222208634018898


In [61]:
example = mnist[0][0]
model(example.unsqueeze(dim=0).cuda())

tensor([[ -6.0502,  -7.1809,  -2.1263,  13.1842, -13.8804,  15.6417,  -7.9835,
           1.2251,   3.4432,   3.7146]], device='cuda:0',
       grad_fn=<AddmmBackward>)